In [20]:
!pip install torch torchvision psutil numpy scipy tqdm click imageio imageio-ffmpeg ninja opensimplex tensorboard --no-cache-dir


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 136.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 165.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 153.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 159.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [18]:
!git clone https://github.com/NVlabs/stylegan3.git
%cd stylegan3
!python train.py --help


Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 212 (delta 99), reused 90 (delta 90), pack-reused 49 (from 1)
Receiving objects: 100% (212/212), 4.16 MiB | 9.75 MiB/s, done.
Resolving deltas: 100% (108/108), done.
/workspace/stylegan3/stylegan3
Usage: train.py [OPTIONS]

  Train a GAN using the techniques described in the paper "Alias-Free
  Generative Adversarial Networks".

  Examples:

  # Train StyleGAN3-T for AFHQv2 using 8 GPUs.
  python train.py --outdir=~/training-runs --cfg=stylegan3-t --data=~/datasets/afhqv2-512x512.zip \
      --gpus=8 --batch=32 --gamma=8.2 --mirror=1

  # Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
  python train.py --outdir=~/training-runs --cfg=stylegan3-r --data=~/datasets/metfacesu-1024x1024.zip \
      --gpus=8 --batch=32 --gamma=6.6 --mirror=1 --kimg=5000 --snap=5 \
    

In [ ]:
import os
import zipfile
import shutil
from PIL import Image
from tqdm import tqdm

base_dir = "/workspace"

zip_path = os.path.join(base_dir, "cat_dataset.zip")
unzipped_dir = os.path.join(base_dir, "cat_dataset_raw")
processed_dir = os.path.join(base_dir, "cat_dataset_64")
output_zip = os.path.join(base_dir, "cat_dataset_64.zip")

if os.path.exists(unzipped_dir):
    shutil.rmtree(unzipped_dir)
os.makedirs(unzipped_dir)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzipped_dir)

if os.path.exists(processed_dir):
    shutil.rmtree(processed_dir)
os.makedirs(processed_dir)

image_paths = []
for root, _, files in os.walk(unzipped_dir):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_paths.append(os.path.join(root, file))

for path in tqdm(image_paths, desc="Resizing images"):
    try:
        filename = os.path.basename(path)
        output_path = os.path.join(processed_dir, filename)

        with Image.open(path) as img:
            img = img.convert("RGB")
            img = img.resize((64, 64))
            img.save(output_path)
    except Exception as e:
        print(f"Skipping {path}: {e}")

with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(processed_dir):
        for file in files:
            full_path = os.path.join(root, file)
            arcname = os.path.relpath(full_path, processed_dir)
            zipf.write(full_path, arcname)


shutil.rmtree(unzipped_dir, ignore_errors=True)
shutil.rmtree(processed_dir, ignore_errors=True)



Resizing images: 100%|██████████| 29843/29843 [06:27<00:00, 76.98it/s] 



✅ Gotowe! ZIP zapisany jako: /workspace/cat_dataset_64.zip
🧹 Wyczyszczono katalogi tymczasowe.


In [5]:
!python stylegan3/train.py \
  --outdir=training-runs \
  --cfg=stylegan3-r \
  --data=cat_dataset_64.zip \
  --gpus=1 \
  --batch=128 \
  --gamma=5.0 \
  --mirror=1 \
  --kimg=250 \
  --tick=10 \
  --snap=5 \
  --resume=/workspace/training-runs/00005-stylegan3-r-cat_dataset_64-gpus1-batch128-gamma5/network-snapshot-000250.pkl


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 65536,
    "channel_max": 1024,
    "magnitude_ema_beta": 0.9955736831881946,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 5.0,
    "blur_init_si

In [3]:
!python stylegan3/metrics.py --help

python: can't open file '/workspace/stylegan3/metrics.py': [Errno 2] No such file or directory


In [34]:
!python stylegan3/calc_metrics.py \
  --gpus=1 \
  --data=cat_dataset_64.zip \
  --metrics=pr50k3 \
  --network=/workspace/training-runs/00002-stylegan3-r-cat_dataset_64-gpus1-batch128-gamma5/network-snapshot-000250.pkl

Loading network from "/workspace/training-runs/00002-stylegan3-r-cat_dataset_64-gpus1-batch128-gamma5/network-snapshot-000250.pkl"...
/usr/local/lib/python3.11/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an is

In [40]:
import torch
import numpy as np
import legacy  # from StyleGAN3 repo
import dnnlib
from torchvision.utils import save_image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

network_pkl = '/workspace/training-runs/00006-stylegan3-r-cat_dataset_64-gpus1-batch128-gamma5/network-snapshot-000250.pkl'
with dnnlib.util.open_url(network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)

z1 = torch.randn([1, G.z_dim], device=device)
z2 = torch.randn([1, G.z_dim], device=device)

w1 = G.mapping(z1, None)
w2 = G.mapping(z2, None)

img1 = G.synthesis(w1, noise_mode='const')
img2 = G.synthesis(w2, noise_mode='const')

def interpolate(w1, w2, steps):
    return [w1 * (1 - t) + w2 * t for t in np.linspace(0, 1, steps)]

interpolated_ws = interpolate(w1, w2, 10)

images = [G.synthesis(w, noise_mode='const') for w in interpolated_ws]
images = torch.cat(images, dim=0)  # [10, C, H, W]

save_image(images, 'interpolation_grid.png', nrow=10, normalize=True, value_range=(-1, 1))

